In [2]:
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


#### import data

In [1]:
# all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
#                 "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
#                 "engaged_with_user_following_count", "engaged_with_user_is_verified",
#                 "engaged_with_user_account_creation",
#                 "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count",
#                 "enaging_user_is_verified",
#                 "enaging_user_account_creation", "engagee_follows_engager"]

# all_features_to_idx = dict(zip(all_features, range(len(all_features))))
# labels_to_idx = {"reply_timestamp": 20, "retweet_timestamp": 21, "retweet_with_comment_timestamp": 22,
#                  "like_timestamp": 23}

# with open("training.tsv", encoding="utf-8") as f:
#     for line in f.readlines()[:5]:
#         line = line.strip()
#         features = line.split("\x01")
#         for feature, idx in all_features_to_idx.items():
#             print("feature {} has value {}".format(feature, features[idx]))

#         for label, idx in labels_to_idx.items():
#             print("label {} has value {}".format(label, features[idx]))


feature text_ tokens has value 101	1942	18628	15752	4458	7697	24309	10634	5618	2395	2598	3584	1946	22480	67316	12434	19054	10634	4704	3350	1910	15752	106975	15355	10083	129	2583	2042	29004	58268	111806	18628	102
feature hashtags has value 
feature tweet_id has value E7D6C5094767223F6F8789A87A1937AB
feature present_media has value 
feature present_links has value 
feature present_domains has value 
feature tweet_type has value TopLevel
feature language has value 22C448FF81263D4BAF2A176145EE9EAD
feature tweet_timestamp has value 1581262691
feature engaged_with_user_id has value D557B03872EF8986F7F4426AE094B2FE
feature engaged_with_user_follower_count has value 986
feature engaged_with_user_following_count has value 1201
feature engaged_with_user_is_verified has value false
feature engaged_with_user_account_creation has value 1274269909
feature enaging_user_id has value 00000776B07587ECA9717BFC301F2D6E
feature enaging_user_follower_count has value 94
feature enaging_user_following_count h

In [2]:
features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
                "engaged_with_user_following_count", "engaged_with_user_is_verified",
                "engaged_with_user_account_creation",
                "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count",
                "engaging_user_is_verified",
                "engaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp", 
                "retweet_with_comment_timestamp", "like_timestamp"]

values = list()
with open("training.tsv", encoding="utf-8") as f:
    for line in tqdm(f.readlines()[:12434735]):  # use 100000 for testing
        line = line.strip()
        data = line.split("\x01")
        values.append(data)
        
df_train = pd.DataFrame(values, columns=features)
print(len(df_train))
df_train.head()


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 106786.48it/s]


100000


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,enaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,101\t1942\t18628\t15752\t4458\t7697\t24309\t10...,,E7D6C5094767223F6F8789A87A1937AB,,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581262691,D557B03872EF8986F7F4426AE094B2FE,...,00000776B07587ECA9717BFC301F2D6E,94,648,false,1478011810,false,,,,
1,101\t56898\t137\t94836\t101481\t10245\t11166\t...,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,129F4A868712BA2B98D31AF98C3066E4,,,,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581497241,424822AC982CE0E8965506C63B44EC12,...,00000B85AAF7DE172876FD96718C4469,1139,46,false,1540395738,true,,1581497559,,1581497622
2,101\t98377\t22627\t33499\t25053\t10898\t3793\t...,,04C6C2175852CDBBC23B2446C7E7C22D,,DDFFB4C01DB85921C3580F614575AA6D,BE4539C53C53FFABCFD232DB100C792B,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580978528,1EC14E26417AA926095530AC591BA9CE,...,00000E0C9B364891CDE89ECFC54771DE,780,440,false,1432084055,true,,,,1581060554
3,101\t56898\t137\t11161\t73421\t131\t137\t11887...,,168157826315514C120494D4DF8E6216,,,,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581321849,9B9595B6FEB8948BDDF0D222F27E0118,...,00000F04EEDBCF3E1FB9A1948BF353B6,1,45,false,1534313747,false,,,,1581328518
4,101\t100\t119\t6694\t6546\t5621\t2446\t1975\t2...,,B3E3673782A69D9D8A45D3B222F0B073,Photo,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580956787,525DC99B7CB8F1AC4AD3E66C53FA38E0,...,000010088197DA00D659853E06935B3E,171,388,false,1490166885,false,,,,1580957807


### feature preprocessing
- using pycharm

#### `prediction target`: turn label to bool value

In [3]:
df_train['isReply'] = np.where(df_train['reply_timestamp']!='', 1, 0)
df_train['isRetweet'] = np.where(df_train['retweet_timestamp']!='', 1, 0)
df_train['isRetweetComment'] = np.where(df_train['retweet_with_comment_timestamp']!='', 1, 0)
df_train['isLike'] = np.where(df_train['like_timestamp']!='', 1, 0)

df_train[df_train['isReply']==1]


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,isReply,isRetweet,isRetweetComment,isLike
6,101\t48993\t10147\t183\t32095\t10104\t190\t133...,,166C053A658691172A7A3CB20D8FB614,Photo\tPhoto\tPhoto,,,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1581004518,A0FD6DF4B4FBF62949708CDB97CC8124,...,1577292324,true,1581008849,,,,1,0,0,0
125,101\t89482\t83684\t11683\t33495\t10908\t10106\...,,F3138860BDA3B5D7F0A737DA7315EAEA,,9068DC6AB9A470B52B756C7E01BB59D5,BBA1D3359A23689C6783652EA73B12BA,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581472802,93AC601D1DC594C1D229741CD46FD64A,...,1438691600,false,1581479241,,,1581479241,1,0,0,1
132,101\t11065\t21852\t117\t10549\t10944\t100\t188...,,8B39E5BCCBE176B89F687C0DCD62625C,,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1580947527,02573D999D611F9F1DA9F29D6A1BF653,...,1201476618,false,1580951299,,,,1,0,0,0
184,101\t20220\t10341\t11140\t112\t187\t57981\t108...,F7C459AC0E2AE073A6D9A1EFFEBF9B74\tB6024D6F02AF...,35741E899D72221E320ED2575C700A5C,Photo\tPhoto\tPhoto\tPhoto,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581306533,F563D23D4C223285F6CFA355D31A1EA0,...,1557127504,false,1581324935,1581324874,,,1,1,0,0
213,101\t56898\t137\t69041\t10325\t10738\t86506\t1...,E2CA178349922542ED02B970D5539C7C,FA26A66ECACCECE8132359AE10E94BA0,GIF,,,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581461936,0DCE53D8795D38E1FA9584238CD2550F,...,1524959218,true,1581466817,1581466761,,1581466761,1,1,0,1
273,101\t100\t19556\t10911\t61362\t11850\t169\t977...,,AC04FA0674B4C836D0F20AEFF84E0844,Photo\tPhoto,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581284342,B4AB5769BDCB8C038AB6CB89CEEB09E1,...,1526946081,true,1581288852,,,,1,0,0,0
312,101\t12489\t169\t92746\t17684\t10269\t102,,9CE55761306656C03E55E2EB5E2FC45D,,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581348258,746C7C00718622E8026B684AD24C9911,...,1313974508,true,1581361980,,,,1,0,0,0
327,101\t100\t5965\t6134\t1946\t2965\t100\t119\t20...,CD33508ADA28A80D3A0DD6728CD7A537\t6F5D51E47F9A...,75229A3B57CCE7FF60C4064837EEDF2C,Photo,F30118EB13346C80D754EAC2067092D5,178B23A1119D4345DA847C10943997F9,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581079406,EF7EF1DEB8DA6C222214280DE31DE790,...,1479563804,true,1581117100,,,,1,0,0,0
329,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,,A61AC31749608EFD5AAC56AC4F68D455,Photo,,,TopLevel,B9175601E87101A984A50F8A62A1C374,1581055817,BE2DD29E89904F3EDFD2BC366E6605F3,...,1411957284,false,1581056245,,,,1,0,0,0
357,101\t156\t14735\t86060\t13530\t10148\t52284\t1...,,5B763E29AD3777B107A394F5F0441FB2,,,,TopLevel,DBEEFB80F8A314311E2B4BD593E11DFE,1581443930,30FF5A8D4937DFCE8FA080039FECBEF3,...,1425770173,true,1581445894,,,1581444123,1,0,0,1


In [4]:
val_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
                "engaged_with_user_following_count", "engaged_with_user_is_verified",
                "engaged_with_user_account_creation",
                "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count",
                "engaging_user_is_verified",
                "engaging_user_account_creation", "engagee_follows_engager"]

values = list()
with open("val.tsv", encoding="utf-8") as f:
    for line in tqdm(f.readlines()[:100000]):
        line = line.strip()
        data = line.split("\x01")
        values.append(data)
        
df_val = pd.DataFrame(values, columns=val_features)
print(len(df_val))
df_val.head()


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 344078.01it/s]


100000


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaged_with_user_account_creation,enaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager
0,101\t47185\t10157\t100986\t10343\t55422\t119\t...,,7647B4E9DAF4C1D8973397DC2A04F3E3,Photo,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581703126,8A9AB92B775C62C4AB60DF6773A01571,13941,1216,false,1448292186,0000006C3074607050F1339DDCB890BB,27448,600,false,1520948869,true
1,101\t56898\t137\t10730\t107105\t59578\t17394\t...,,408DB1803264B5FF55F73EC06BE9BD77,Photo\tPhoto\tPhoto\tPhoto,,,Retweet,B9175601E87101A984A50F8A62A1C374,1582021842,825B003667DF15692B1704F44DAFD1A2,186,100,false,1263078566,000013315386492275CCBF7AEF293EF0,139,956,false,1335110299,false
2,101\t16735\t11584\t131\t14120\t131\t120\t120\t...,,2EE951379C47E8BF62EABB8FA027F753,GIF,,,TopLevel,2996EB2FE8162C076D070A4C8D6532CD,1581734918,2DEC09796D0EB473244CB7A4FB13A0A0,250470,1,false,1356488269,00001569CB28972FC8173122D9DA162F,16,97,false,1503940711,false
3,101\t56898\t137\t12127\t43213\t10245\t168\t102...,,2135F24B05DAE3EF213F9CE80FDC6DAF,Photo\tPhoto\tPhoto,,,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581913613,487BCB8F1677A6D9EEE612940410CBB6,516,406,false,1449096567,00001607209C5774DF9207A2AC0EED5F,460,693,false,1396311956,true
4,101\t11101\t42780\t10741\t11222\t10197\t15304\...,,09143FEDE9BD494A6EA9A7EE160565E3,Photo,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581565745,C85C39DC8D279E51DF1BBF0391E5EB2E,19576,273,true,1236181798,0000177705514C315F2FC6DFA3872712,468,3837,false,1298646801,false


In [1]:
val_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
                "engaged_with_user_following_count", "engaged_with_user_is_verified",
                "engaged_with_user_account_creation",
                "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count",
                "engaging_user_is_verified",
                "engaging_user_account_creation", "engagee_follows_engager"]

values = list()
with open("val_v2.tsv", encoding="utf-8") as f:
    for line in tqdm(f.readlines()):
        line = line.strip()
        data = line.split("\x01")
        values.append(data)
        
df_val = pd.DataFrame(values, columns=val_features)
print(len(df_val))
df_val.head()


NameError: name 'tqdm' is not defined

#### feature engineering: `text_tokens`
1. `d2v__text_tokens`: to dense using `doc2vec`

In [5]:
documents = [TaggedDocument(doc.split('\t'), [i]) for i, doc in enumerate(df_train['text_tokens'].values)]
model = Doc2Vec(documents, vector_size=10, window=2, min_count=1, workers=4)

# store to local
fname = get_tmpfile("doc2vec_5x_val")
model.save(fname)


In [5]:
# load model
fname = get_tmpfile("doc2vec_5x_val")
model = Doc2Vec.load(fname)


In [32]:
# load text_tokens doc2vec model
fname = get_tmpfile("doc2vec_5x_val")
model_texttokens = Doc2Vec.load(fname)

val_text_vector = list()
for i in tqdm(range(0, len(df_val))):
    val_text_vector.append(model_texttokens.infer_vector(df_val['text_tokens'].values[i].split('\t')))
df_val_text_vectors = pd.DataFrame(val_text_vector)
df_val_text_vectors


# create doc2vec dense feature for train.tsv
train_text_vecs = list()
for i in tqdm(range(0, len(df_train))):
    train_text_vecs.append(model_texttokens.infer_vector(df_train['text_tokens'].values[i].split('\t')))
df_train_text_vectors = pd.DataFrame(train_text_vecs)
df_train_text_vectors

df_train = pd.concat([df_train, df_train_text_vectors], axis=1)
df_val = pd.concat([df_val, df_val_text_vectors], axis=1)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12434735/12434735 [2:23:18<00:00, 1446.13it/s]


12434735


#### feature engineering: `hashtags`
- no, or multiple tags
- seems encode as another type
So,
1. `isHashtag`
2. `d2v__hashtag`


In [9]:
# 1. `isHashtag`
df_train['isHashtag'] = np.where(df_train['hashtags'] != '', 1, 0)

# 2. d2v__hashtags
hashtags_documents = [TaggedDocument(doc.split('\t'), [i]) for i, doc in enumerate(df_train[df_train['isHashtag']==1]['hashtags'].values)]
model_hashtag = Doc2Vec(hashtags_documents, vector_size=10, window=2, min_count=1, workers=4)
# store to local
fname = get_tmpfile("doc2vec_hashtags")
model_hashtag.save(fname)
model_hashtag = Doc2Vec.load(fname)
print('model_hashtag has saved and loaded.')


d2v_model_hashtags has saved and loaded.


In [22]:
# load hashtag doc2vec model
fname = get_tmpfile("doc2vec_hashtags")
model_hashtag = Doc2Vec.load(fname)

val_hashtag_vector = list()
for i in tqdm(range(0, len(df_val))):
    if df_val['hashtags'].values[i] == 'nan':
        val_hashtag_vector.append([0,0,0,0,0])
    else:
        val_hashtag_vector.append(model_hashtag.infer_vector(df_val['hashtags'].values[i].split('\t')))
df_val_hashtag_vectors = pd.DataFrame(val_hashtag_vector)
df_val_hashtag_vectors


# create doc2vec dense feature for train.tsv
train_hashtag_vecs = list()
for i in tqdm(range(0, len(df_train))):
    if df_train['hashtags'].values[i] == 'nan':
        train_hashtag_vecs.append([0,0,0,0,0])
    else:
        train_hashtag_vecs.append(model_hashtag.infer_vector(df_train['hashtags'].values[i].split('\t')))
df_train_hashtag_vectors = pd.DataFrame(train_hashtag_vecs)
df_train_hashtag_vectors

df_train = pd.concat([df_train, df_train_hashtag_vectors], axis=1)
df_val = pd.concat([df_val, df_val_hashtag_vectors], axis=1)


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,isReply,isRetweet,isRetweetComment,isLike,isHashtag,hashtags_vector
1,101\t56898\t137\t94836\t101481\t10245\t11166\t...,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,129F4A868712BA2B98D31AF98C3066E4,,,,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581497241,424822AC982CE0E8965506C63B44EC12,...,,1581497559,,1581497622,0,1,0,1,1,"[-0.0013035543, 0.0307784, 0.005114223, -0.001..."
5,101\t13498\t789\t45935\t19721\t10700\t93914\t7...,D20331F40622336B266EC2BF3572F7E5,AB21A06B694D637075F1EA4F89A05197,Video,,,TopLevel,167115458A0DBDFF7E9C0C53A83BAC9B,1581341389,7E1E2FAD93219D0247BDBE451AB343E9,...,,1581346588,,1581346588,0,1,0,1,1,"[-0.039649453, -0.0047600097, 0.049533647, 0.0..."
10,101\t18302\t24747\t10929\t196\t108\t102549\t30...,D545FDCEEE2C82E21EBBE23505173FA0,311EDE393CEBB5E880F5B3A96A69AA94,Photo,,,TopLevel,4DC22C3F31C5C43721E6B5815A595ED6,1581187934,85A39142470D65A77BB9E86B054AD321,...,,,,1581189873,0,0,0,1,1,"[0.0133725945, -0.047275785, -0.03791277, 0.01..."
12,101\t12845\t84558\t74874\t10135\t20442\t17330\...,40A23C9DE38F5B42FDABD7DE6B73AC6E,EC2126D5DB025A6C66BC24A5596EC475,,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581043204,46097861C196B25F2C56606CD32AE14C,...,,,,1581045318,0,0,0,1,1,"[0.01189985, 0.028408742, -0.033036105, 0.0029..."
14,101\t122\t4460\t10365\t4348\t8133\t2384\t1890\...,C321B459658CB455C53F2DE7C235A706\t34A45F5ED2E4...,86E049967C94CF82BA9EB17A2EE4F3CE,Photo\tPhoto\tPhoto,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581062422,49D04A274A357E3039E1DF0F78E2975F,...,,,,1581063697,0,0,0,1,1,"[0.027290443, 0.02467559, 0.01614068, -0.03772..."
16,101\t10117\t54963\t10124\t10472\t10491\t10479\...,56402281D4D3E8F951CB23AC700A8F5C,9CF609C0D9D099F7C09B52976F5029F7,,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581017422,A702303ED1A25C6DD1393BC0CCABF94A,...,,1581017998,,1581017998,0,1,0,1,1,"[0.020094339, 0.016461954, 0.03864599, -0.0205..."
17,101\t63537\t10114\t27541\t10105\t80914\t137\t1...,805C7A95714ADBA122FF5F55C9C9C8E4\t6372CE4C7D66...,6601197539548AAD5C9CB9FCD162A5C7,Photo\tPhoto,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581521058,0FACF6DB63422FC388A2BB4AA1585AB2,...,,,,,0,0,0,0,1,"[0.026069092, 0.02858966, -0.03593265, 0.03529..."
18,101\t56898\t137\t51551\t15529\t10246\t19360\t1...,D8B2515734BFAA8902070F3A9C4F6BCD,B72C9FD782AF79BA56755410E3C617FC,Video,,,Retweet,06D61DCBBE938971E1EA0C38BD9B5446,1581081184,0D01AD1116E22830152CC5FC78CE952D,...,,,,,0,0,0,0,1,"[-0.039616328, -0.01595917, -0.019717477, 0.00..."
23,101\t10576\t10105\t41008\t10106\t10531\t28757\...,F9B5807EDB6B215DF96862159A6B4892,C83B3441704D4E6172B86CEF3F66845C,Photo,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581147754,5611A5DB21B6C40C8E3848A4DECFBB24,...,,,,,0,0,0,0,1,"[-0.0003065907, -0.02090835, -0.025122842, -0...."
27,101\t108\t1913\t61495\t18628\t10634\t28224\t18...,57BDABF6E201A2D0B6C7652379FABEEE,28D9CAE78D1F3E9877A9559757D57293,,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580954292,5A6DCA175E07A222854B7F115C50A3B8,...,,1580955948,,1580955948,0,1,0,1,1,"[0.01707836, -0.009136748, -0.00038563312, 0.0..."


#### feature engineering: `present_media`
- has 3 type mixing: Photo, GIF, Video
So,
1. make a vector to describe the media presents: [# of Photo, # of GIF, # of Video]


In [31]:
# df_train.groupby(['present_media']).size()
list__present_media = list()
for item in tqdm(df_train['present_media'].values):
    list_tmp_1 = [item.split('\t').count('Photo'), item.split('\t').count('GIF'), item.split('\t').count('Video')]
    list__present_media.append(list_tmp_1)

df_train_vec__present_media = pd.DataFrame(list__present_media)
df_train_vec__present_media.head()

list__present_media = list()
for item in tqdm(df_val['present_media'].values):
    list_tmp_1 = [item.split('\t').count('Photo'), item.split('\t').count('GIF'), item.split('\t').count('Video')]
    list__present_media.append(list_tmp_1)

df_val_vec__present_media = pd.DataFrame(list__present_media)
df_val_vec__present_media.head()

df_train = pd.concat([df_train, df_train_vec__present_media], axis=1)
df_val = pd.concat([df_val, df_val_vec__present_media], axis=1)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 315614.64it/s]


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,isReply,isRetweet,isRetweetComment,isLike,isHashtag,hashtags_vector,vec__present_media
1,101\t56898\t137\t94836\t101481\t10245\t11166\t...,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,129F4A868712BA2B98D31AF98C3066E4,,,,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581497241,424822AC982CE0E8965506C63B44EC12,...,1581497559,,1581497622,0,1,0,1,1,"[-0.0013035543, 0.0307784, 0.005114223, -0.001...","[0, 0, 0]"
5,101\t13498\t789\t45935\t19721\t10700\t93914\t7...,D20331F40622336B266EC2BF3572F7E5,AB21A06B694D637075F1EA4F89A05197,Video,,,TopLevel,167115458A0DBDFF7E9C0C53A83BAC9B,1581341389,7E1E2FAD93219D0247BDBE451AB343E9,...,1581346588,,1581346588,0,1,0,1,1,"[-0.039649453, -0.0047600097, 0.049533647, 0.0...","[0, 0, 1]"
10,101\t18302\t24747\t10929\t196\t108\t102549\t30...,D545FDCEEE2C82E21EBBE23505173FA0,311EDE393CEBB5E880F5B3A96A69AA94,Photo,,,TopLevel,4DC22C3F31C5C43721E6B5815A595ED6,1581187934,85A39142470D65A77BB9E86B054AD321,...,,,1581189873,0,0,0,1,1,"[0.0133725945, -0.047275785, -0.03791277, 0.01...","[1, 0, 0]"
12,101\t12845\t84558\t74874\t10135\t20442\t17330\...,40A23C9DE38F5B42FDABD7DE6B73AC6E,EC2126D5DB025A6C66BC24A5596EC475,,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581043204,46097861C196B25F2C56606CD32AE14C,...,,,1581045318,0,0,0,1,1,"[0.01189985, 0.028408742, -0.033036105, 0.0029...","[0, 0, 0]"
14,101\t122\t4460\t10365\t4348\t8133\t2384\t1890\...,C321B459658CB455C53F2DE7C235A706\t34A45F5ED2E4...,86E049967C94CF82BA9EB17A2EE4F3CE,Photo\tPhoto\tPhoto,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581062422,49D04A274A357E3039E1DF0F78E2975F,...,,,1581063697,0,0,0,1,1,"[0.027290443, 0.02467559, 0.01614068, -0.03772...","[3, 0, 0]"


#### feature engineering: `present_links` & `present_domains`
- seldomly repeat between links, so make boolean
- relatively more domains are repeated, so make a weighted value with log2 function


In [115]:
df_tmp_3 = df_train.groupby(['present_domains']).size().reset_index()
df_tmp_3 = df_tmp_3[df_tmp_3['present_domains']!='']
df_tmp_3.rename(columns={0: 'weighted__domains'}, inplace=True)
df_train = pd.merge(df_train, df_tmp_3, on='present_domains', how='left')
df_train = df_train.fillna(0)
df_train['weighted__domains'] = df_train['weighted__domains'].apply(lambda x: math.log(x+1, 2))  # x+1 to prevent from log function error

df_train['isPresent_links'] = np.where(df_train['present_links']!='', 1, 0)
df_train.head()


#### feature engineering: `tweet_type`
- countable classes, so make onehot


In [69]:
df_train.groupby(['tweet_type']).size()
df_tmp_4 = pd.get_dummies(df_train[['tweet_type']])
df_tmp_4.drop('tweet_type_Quote', axis=1, inplace=True)
df_tmp_4

df_train = pd.concat([df_train, df_tmp_4], axis=1)
df_train


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,isRetweet,isRetweetComment,isLike,isHashtag,hashtags_vector,vec__present_media,domain_freq,weighted__domains,tweet_type_Retweet,tweet_type_TopLevel
1,101\t56898\t137\t94836\t101481\t10245\t11166\t...,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,129F4A868712BA2B98D31AF98C3066E4,,,,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581497241,424822AC982CE0E8965506C63B44EC12,...,1,0,1,1,"[-0.0013035543, 0.0307784, 0.005114223, -0.001...","[0, 0, 0]",2.0,1.584963,1,0
5,101\t13498\t789\t45935\t19721\t10700\t93914\t7...,D20331F40622336B266EC2BF3572F7E5,AB21A06B694D637075F1EA4F89A05197,Video,,,TopLevel,167115458A0DBDFF7E9C0C53A83BAC9B,1581341389,7E1E2FAD93219D0247BDBE451AB343E9,...,1,0,1,1,"[-0.039649453, -0.0047600097, 0.049533647, 0.0...","[0, 0, 1]",1.0,1.000000,0,1
10,101\t18302\t24747\t10929\t196\t108\t102549\t30...,D545FDCEEE2C82E21EBBE23505173FA0,311EDE393CEBB5E880F5B3A96A69AA94,Photo,,,TopLevel,4DC22C3F31C5C43721E6B5815A595ED6,1581187934,85A39142470D65A77BB9E86B054AD321,...,0,0,1,1,"[0.0133725945, -0.047275785, -0.03791277, 0.01...","[1, 0, 0]",1.0,1.000000,0,1
12,101\t12845\t84558\t74874\t10135\t20442\t17330\...,40A23C9DE38F5B42FDABD7DE6B73AC6E,EC2126D5DB025A6C66BC24A5596EC475,,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581043204,46097861C196B25F2C56606CD32AE14C,...,0,0,1,1,"[0.01189985, 0.028408742, -0.033036105, 0.0029...","[0, 0, 0]",2.0,1.584963,0,0
14,101\t122\t4460\t10365\t4348\t8133\t2384\t1890\...,C321B459658CB455C53F2DE7C235A706\t34A45F5ED2E4...,86E049967C94CF82BA9EB17A2EE4F3CE,Photo\tPhoto\tPhoto,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581062422,49D04A274A357E3039E1DF0F78E2975F,...,0,0,1,1,"[0.027290443, 0.02467559, 0.01614068, -0.03772...","[3, 0, 0]",1.0,1.000000,0,1
16,101\t10117\t54963\t10124\t10472\t10491\t10479\...,56402281D4D3E8F951CB23AC700A8F5C,9CF609C0D9D099F7C09B52976F5029F7,,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581017422,A702303ED1A25C6DD1393BC0CCABF94A,...,1,0,1,1,"[0.020094339, 0.016461954, 0.03864599, -0.0205...","[0, 0, 0]",1.0,1.000000,0,1
17,101\t63537\t10114\t27541\t10105\t80914\t137\t1...,805C7A95714ADBA122FF5F55C9C9C8E4\t6372CE4C7D66...,6601197539548AAD5C9CB9FCD162A5C7,Photo\tPhoto,,,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581521058,0FACF6DB63422FC388A2BB4AA1585AB2,...,0,0,0,1,"[0.026069092, 0.02858966, -0.03593265, 0.03529...","[2, 0, 0]",2.0,1.584963,0,1
18,101\t56898\t137\t51551\t15529\t10246\t19360\t1...,D8B2515734BFAA8902070F3A9C4F6BCD,B72C9FD782AF79BA56755410E3C617FC,Video,,,Retweet,06D61DCBBE938971E1EA0C38BD9B5446,1581081184,0D01AD1116E22830152CC5FC78CE952D,...,0,0,0,1,"[-0.039616328, -0.01595917, -0.019717477, 0.00...","[0, 0, 1]",6.0,2.807355,1,0
23,101\t10576\t10105\t41008\t10106\t10531\t28757\...,F9B5807EDB6B215DF96862159A6B4892,C83B3441704D4E6172B86CEF3F66845C,Photo,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581147754,5611A5DB21B6C40C8E3848A4DECFBB24,...,0,0,0,1,"[-0.0003065907, -0.02090835, -0.025122842, -0....","[1, 0, 0]",1.0,1.000000,0,0
27,101\t108\t1913\t61495\t18628\t10634\t28224\t18...,57BDABF6E201A2D0B6C7652379FABEEE,28D9CAE78D1F3E9877A9559757D57293,,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580954292,5A6DCA175E07A222854B7F115C50A3B8,...,1,0,1,1,"[0.01707836, -0.009136748, -0.00038563312, 0.0...","[0, 0, 0]",1.0,1.000000,0,1


#### feature engineering: `difference__`
- see if engagements happens when followers_count of engaged user is higher than engaging user
    - `engaged_with_user_follower_count`- `engaged_with_user_following_count`
    - `enaging_user_follower_count` - `enaging_user_following_count`
- turn boolean: `engaged_with_user_is_verified`, `enaging_user_is_verified`, `engagee_follows_engager`


In [111]:
selected_columns = ['engaged_with_user_follower_count','engaged_with_user_following_count','enaging_user_follower_count','enaging_user_following_count']
for c in selected_columns:
    df_train[c] = df_train[c].astype(int)

df_train['difference__engaged_user_follow_count'] = df_train['engaged_with_user_follower_count'] - df_train['engaged_with_user_following_count']
df_train['difference__engaging_user_follow_count'] = df_train['enaging_user_follower_count'] - df_train['enaging_user_following_count']
df_train['difference__followers_count'] = df_train['engaged_with_user_follower_count'] - df_train['enaging_user_follower_count']
df_train['difference__following_count'] = df_train['engaged_with_user_following_count'] - df_train['enaging_user_following_count']
df_train.head()


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,isHashtag,hashtags_vector,vec__present_media,tweet_type_Retweet,tweet_type_TopLevel,weighted__domains,difference__engaged_user_follow_count,difference__engaging_user_follow_count,difference__followers_count,difference__following_count
0,101\t56898\t137\t94836\t101481\t10245\t11166\t...,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,129F4A868712BA2B98D31AF98C3066E4,,,,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581497241,424822AC982CE0E8965506C63B44EC12,...,1,"[-0.0013035543, 0.0307784, 0.005114223, -0.001...","[0, 0, 0]",1,0,0.0,548,1093,86,631
1,101\t13498\t789\t45935\t19721\t10700\t93914\t7...,D20331F40622336B266EC2BF3572F7E5,AB21A06B694D637075F1EA4F89A05197,Video,,,TopLevel,167115458A0DBDFF7E9C0C53A83BAC9B,1581341389,7E1E2FAD93219D0247BDBE451AB343E9,...,1,"[-0.039649453, -0.0047600097, 0.049533647, 0.0...","[0, 0, 1]",0,1,0.0,46248,513,45751,16
2,101\t18302\t24747\t10929\t196\t108\t102549\t30...,D545FDCEEE2C82E21EBBE23505173FA0,311EDE393CEBB5E880F5B3A96A69AA94,Photo,,,TopLevel,4DC22C3F31C5C43721E6B5815A595ED6,1581187934,85A39142470D65A77BB9E86B054AD321,...,1,"[0.0133725945, -0.047275785, -0.03791277, 0.01...","[1, 0, 0]",0,1,0.0,19220,-110,19350,20
3,101\t12845\t84558\t74874\t10135\t20442\t17330\...,40A23C9DE38F5B42FDABD7DE6B73AC6E,EC2126D5DB025A6C66BC24A5596EC475,,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581043204,46097861C196B25F2C56606CD32AE14C,...,1,"[0.01189985, 0.028408742, -0.033036105, 0.0029...","[0, 0, 0]",0,0,0.0,159345,-229,159548,-26
4,101\t122\t4460\t10365\t4348\t8133\t2384\t1890\...,C321B459658CB455C53F2DE7C235A706\t34A45F5ED2E4...,86E049967C94CF82BA9EB17A2EE4F3CE,Photo\tPhoto\tPhoto,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581062422,49D04A274A357E3039E1DF0F78E2975F,...,1,"[0.027290443, 0.02467559, 0.01614068, -0.03772...","[3, 0, 0]",0,1,0.0,82249,-71,82257,-63


In [102]:
df_train['engaged_with_user_is_verified'] = np.where(df_train['engaged_with_user_is_verified']=='true', 1, 0)
df_train['enaging_user_is_verified'] = np.where(df_train['enaging_user_is_verified']=='true', 1, 0)
df_train['engagee_follows_engager'] = np.where(df_train['engagee_follows_engager']=='true', 1, 0)


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,isReply,isRetweet,isRetweetComment,isLike,isHashtag,hashtags_vector,vec__present_media,tweet_type_Retweet,tweet_type_TopLevel,weighted__domains
0,101\t56898\t137\t94836\t101481\t10245\t11166\t...,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,129F4A868712BA2B98D31AF98C3066E4,,,,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581497241,424822AC982CE0E8965506C63B44EC12,...,0,1,0,1,1,"[-0.0013035543, 0.0307784, 0.005114223, -0.001...","[0, 0, 0]",1,0,0.0
1,101\t13498\t789\t45935\t19721\t10700\t93914\t7...,D20331F40622336B266EC2BF3572F7E5,AB21A06B694D637075F1EA4F89A05197,Video,,,TopLevel,167115458A0DBDFF7E9C0C53A83BAC9B,1581341389,7E1E2FAD93219D0247BDBE451AB343E9,...,0,1,0,1,1,"[-0.039649453, -0.0047600097, 0.049533647, 0.0...","[0, 0, 1]",0,1,0.0
2,101\t18302\t24747\t10929\t196\t108\t102549\t30...,D545FDCEEE2C82E21EBBE23505173FA0,311EDE393CEBB5E880F5B3A96A69AA94,Photo,,,TopLevel,4DC22C3F31C5C43721E6B5815A595ED6,1581187934,85A39142470D65A77BB9E86B054AD321,...,0,0,0,1,1,"[0.0133725945, -0.047275785, -0.03791277, 0.01...","[1, 0, 0]",0,1,0.0
3,101\t12845\t84558\t74874\t10135\t20442\t17330\...,40A23C9DE38F5B42FDABD7DE6B73AC6E,EC2126D5DB025A6C66BC24A5596EC475,,,,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581043204,46097861C196B25F2C56606CD32AE14C,...,0,0,0,1,1,"[0.01189985, 0.028408742, -0.033036105, 0.0029...","[0, 0, 0]",0,0,0.0
4,101\t122\t4460\t10365\t4348\t8133\t2384\t1890\...,C321B459658CB455C53F2DE7C235A706\t34A45F5ED2E4...,86E049967C94CF82BA9EB17A2EE4F3CE,Photo\tPhoto\tPhoto,,,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581062422,49D04A274A357E3039E1DF0F78E2975F,...,0,0,0,1,1,"[0.027290443, 0.02467559, 0.01614068, -0.03772...","[3, 0, 0]",0,1,0.0


#### 暫存一份

In [8]:
df_train.to_csv('df_train.csv')

left for consideration:
- language
- 